In [5]:
# main_optuna.py
import os
from data_pipeline import setup_tensorflow, create_data_pipeline, get_class_weights_from_dataset
from optuna_optimizer import (run_optuna_optimization, create_final_model, 
                            train_final_model, evaluate_model, visualize_optuna_results)
from visualization import (plot_training_history, safe_confusion_matrix_analysis, 
                         test_random_predictions_modern)

def main():
    # Настройка
    setup_tensorflow()
    
    # Пути
    IMAGE_PATH = "/home/oleg/projects/datasets/construction"
    print(f"Содержимое папки: {os.listdir(IMAGE_PATH)}")
    
    # Создание пайплайна
    train_data, validation_data, train_data_raw, class_names, num_classes = create_data_pipeline(
        image_path=IMAGE_PATH,
        height=224,
        width=224, 
        batch_size=32
    )
    
    # Расчет весов классов
    print("🎯 Расчет весов классов...")
    class_weights = get_class_weights_from_dataset(train_data_raw, class_names)
    
    # Оптимизация Optuna
    study = run_optuna_optimization(
        train_data=train_data,
        validation_data=validation_data,
        num_classes=num_classes,
        model_name='EfficientNetV2S',
        n_trials=20
    )
    
    # Визуализация результатов Optuna
    visualize_optuna_results(study)
    
    # Создание финальной модели
    best_model = create_final_model(study, num_classes)
    
    # Обучение финальной модели
    final_history = train_final_model(
        model=best_model,
        train_data=train_data,
        validation_data=validation_data,
        class_weights=class_weights,
        epochs=70
    )
    
    # Оценка модели
    val_loss, val_accuracy = evaluate_model(best_model, validation_data, "trained_optuna_model")
    
    # Визуализация результатов
    plot_training_history(final_history)
    safe_confusion_matrix_analysis(best_model, validation_data, class_names)
    test_random_predictions_modern(best_model, validation_data, class_names)
    
    print("🎉 ВСЕ ЗАДАЧИ ВЫПОЛНЕНЫ!")

if __name__ == "__main__":
    main()

ImportError: cannot import name 'run_optuna_optimization' from 'optuna_optimizer' (/home/oleg/projects/ML/app/optuna_optimizer.py)

In [3]:
# Настройка и создание пайплайна
setup_tensorflow()
train_data, validation_data, train_data_raw, class_names, num_classes = create_data_pipeline(IMAGE_PATH)

🔄 СОЗДАЕМ ПАЙПЛАЙН...
Found 4990 files belonging to 8 classes.
Using 3992 files for training.
Found 4990 files belonging to 8 classes.
Using 998 files for validation.
Labels shape: (32,)
Labels: [0 5 1 5 5 6 2 6 6 0]
🏷️ Классы: ['bulldozer', 'car', 'dump_truck', 'excavator', 'ice_rink', 'mining_loader', 'person', 'truck_crane']
🔍 Проверка препроцессинга:
Shape: (32, 224, 224, 3)
Data range: 0.000 to 255.000
Mean: 112.395
✅ Пайплайн создан!


2025-10-25 20:01:09.952428: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [2]:
# Создание и обучение модели
model = create_model(num_classes)
history = train_model(model, train_data, validation_data, class_weights)

NameError: name 'create_model' is not defined